# Modelling_Geotab Intersection Congestion 

In [1]:
__author__ = "Zhiji Ding"
__copyright__ = "Copyright 2018, Zhiji Ding"
__email__ = "jimmydzj2006@gmail.com"

In [2]:
# Module import
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib as matplot
import seaborn as sns
%matplotlib inline

In [3]:
# Load data
train_df=pd.read_csv('/Users/jimmyding/Downloads/bigquery-geotab-intersection-congestion/train_final.csv')
test_df=pd.read_csv('/Users/jimmyding/Downloads/bigquery-geotab-intersection-congestion/test_final.csv')

In [4]:
train_df.columns.to_list()

['Unnamed: 0',
 'EntryStreetName',
 'ExitStreetName',
 'EntryHeading',
 'ExitHeading',
 'Weekend',
 'TotalTimeStopped_p20',
 'TotalTimeStopped_p40',
 'TotalTimeStopped_p50',
 'TotalTimeStopped_p60',
 'TotalTimeStopped_p80',
 'DistanceToFirstStop_p20',
 'DistanceToFirstStop_p40',
 'DistanceToFirstStop_p50',
 'DistanceToFirstStop_p60',
 'DistanceToFirstStop_p80',
 'DriveDirection',
 'Distance',
 'Hour_Day',
 'Hour_Night',
 'Hour_Rush',
 'City_Atlanta',
 'City_Boston',
 'City_Chicago',
 'City_Philadelphia']

In [5]:
X = train_df[['EntryStreetName',
 'ExitStreetName',
 'EntryHeading',
 'ExitHeading',
 'Weekend',
 'DriveDirection',
 'Distance',
 'Hour_Day',
 'Hour_Night',
 'Hour_Rush',
 'City_Atlanta',
 'City_Boston',
 'City_Chicago',
 'City_Philadelphia']]

y1=train_df['DistanceToFirstStop_p20']
y2=train_df['DistanceToFirstStop_p50']
y3=train_df['DistanceToFirstStop_p80']
y4=train_df['TotalTimeStopped_p20']
y5=train_df['TotalTimeStopped_p50']
y6=train_df['TotalTimeStopped_p80']

In [6]:
X.head()

,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Weekend,DriveDirection,Distance,Hour_Day,Hour_Night,Hour_Rush,City_Atlanta,City_Boston,City_Chicago,City_Philadelphia
0,2.0,2.0,1.75,1.75,0,0.0,0.287816,0,1,0,1,0,0,0
1,2.0,2.0,0.75,0.75,0,0.0,0.287816,0,1,0,1,0,0,0
2,2.0,2.0,1.75,1.75,0,0.0,0.287816,0,1,0,1,0,0,0
3,2.0,2.0,0.75,0.75,0,0.0,0.287816,0,1,0,1,0,0,0
4,2.0,2.0,1.75,1.75,0,0.0,0.287816,0,1,0,1,0,0,0


In [7]:
test_df=test_df[['EntryStreetName',
 'ExitStreetName',
 'EntryHeading',
 'ExitHeading',
 'Weekend',
 'DriveDirection',
 'Distance',
 'Hour_Day',
 'Hour_Night',
 'Hour_Rush',
 'City_Atlanta',
 'City_Boston',
 'City_Chicago',
 'City_Philadelphia']]
test_df.head()

,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Weekend,DriveDirection,Distance,Hour_Day,Hour_Night,Hour_Rush,City_Atlanta,City_Boston,City_Chicago,City_Philadelphia
0,1.0,1.0,1.25,0.75,0,-0.5,0.038314,0,1,0,1,0,0,0
1,1.0,1.0,1.25,1.25,0,0.0,0.038314,0,1,0,1,0,0,0
2,1.0,1.0,0.25,0.25,0,0.0,0.038314,0,1,0,1,0,0,0
3,1.0,1.0,1.25,1.25,0,0.0,0.038314,0,1,0,1,0,0,0
4,1.0,1.0,0.25,0.25,0,0.0,0.038314,0,1,0,1,0,0,0


# Random Forest Model using grid search

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel

In [ ]:
rf=RandomForestRegressor()
param_grid = {
    'n_estimators':[10,200,800],
    'max_depth':[5,10,None],
    'max_features':[3,6,10],
    'min_samples_split': [4,10,14],
    'min_samples_leaf': [2,3,5]
}
rf_model = GridSearchCV(rf, param_grid, cv=10, n_jobs=1, scoring='neg_mean_squared_error')
rf_model.fit(X, y1)

print("Best parameter (CV %RMSE):" % sqrt(rf_model.best_score_))
print(rf_model.best_params_)

# LightGBM, catboostRegressor and XGBoost with Bayesien Optimization
Reference: https://medium.com/analytics-vidhya/hyperparameters-optimization-for-lightgbm-catboost-and-xgboost-regressors-using-bayesian-6e7c495947a9

## 1.LightGBM

In [ ]:
import lightgbm as lgb
import catboost as cgb
import xgboost as xgb
from bayes_opt import BayesianOptimization
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def bayesion_opt_lgbm(X, y, init_iter=3, n_iters=7, random_state=11, seed = 101, num_iterations = 100):
    dtrain = lgb.Dataset(data=X, label=y)
    # Objective Function
    def hyp_lgbm(num_leaves, feature_fraction, bagging_fraction, max_depth, min_split_gain, min_child_weight):
        
        params = {'application':'regression',
                  'num_iterations': num_iterations,
                  'learning_rate':0.05, 
                  'early_stopping_round':50,
                  'metric':'rmse'} # Default parameters
        params['num_leaves'] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_results = lgb.cv(params, dtrain, nfold=5, seed=seed,categorical_feature=[], stratified=False,
                              verbose_eval =None, feval=rmse)
          # print(cv_results)
        return np.max(cv_results['rmse'])

    # Search space
    pds = {'num_leaves': (120, 230),
          'feature_fraction': (0.3, 0.9),
          'bagging_fraction': (0.8, 1),
          'max_depth': (8, 19),
          'min_split_gain': (0.001, 0.1),
          'min_child_weight': (1, 20)
          }

    # Surrogate model
    optimizer = BayesianOptimization(hyp_lgbm,pds,random_state=7)
                                  
    # Optimize
    optimizer.maximize(init_points=5, n_iter=12)


In [ ]:
bayesion_opt_lgbm(X, y1, init_iter=5, n_iters=10, random_state=77, seed = 101, num_iterations = 200)

# 2.Catboost regressor

In [ ]:
# Objective function
def cat_hyp(depth, bagging_temperature): # Function to optimize depth and bagging temperature
    params = {"iterations": 100,
            "learning_rate": 0.05,
            "eval_metric": "R2",
            "verbose": False} # Default Parameters
    params[ "depth"] = int(round(depth)) 
    params["bagging_temperature"] = bagging_temperature
  
    cat_feat = [] # Categorical features list
    cv_dataset = cgb.Pool(data=X,
                  label=y1,
                  cat_features=cat_feat)

    scores = cgb.cv(cv_dataset,
              params,
              fold_count=3)
    return np.max(scores['rmse'])  # Return maximum RMSE

# Search space
pds = {'depth': (5, 8),
          'bagging_temperature': (3,10)
          }

# Surrogate model
optimizer = BayesianOptimization(cat_hyp, pds, random_state=2100)

# Optimize
optimizer.maximize(init_points=3, n_iter=7)

# 3. XGBoost regressor

In [ ]:
dtrain = xgb.DMatrix(X, y1, feature_names=X.columns.values)

# Objective function
def hyp_xgb(max_depth, subsample, colsample_bytree,min_child_weight, gamma ):
    params = {
    'n_estimators': 300,
    'eta': 0.05,
    'objective': 'reg:linear',
    'eval_metric':'rmse', 
    'silent': 1
     }
    params['max_depth'] = int(round(max_depth))
    params['subsample'] = max(min(subsample, 1), 0)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['min_child_weight'] = int(min_child_weight)
    params['gamma'] = max(gamma, 0)
    scores = xgb.cv(params, dtrain, num_boost_round=1000,verbose_eval=False, early_stopping_rounds=10, feval=xgb_r2, maximize=True, nfold=5)
    return  scores['rmse'].iloc[-1]

# Search space
pds ={
  'min_child_weight':(14, 20),
  'gamma':(0, 5),
  'subsample':(0.5, 1),
  'colsample_bytree':(0.1, 1),
  'max_depth': (5, 10)
}

# Surrogate model
optimizer = BayesianOptimization(hyp_xgb, pds, random_state=10)

# Optimize
optimizer.maximize(init_points=5, n_iter=15)
